In [2]:
#read the data
import pandas as ps
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import math



In [21]:
train_path = 'data/train.csv'
test_path = 'data/test.csv'

train = ps.read_csv(train_path)
test = ps.read_csv(test_path)

test.head()

,id,tweet,state,location
0,4,Edinburgh peeps is it sunny?? #weather,NaN,birmingham
1,5,"SEEVERE T’STORM WARNING FOR TROUSDALE, NORTHW...",NaN,Nashville
2,7,@Agilis1 sport or traditional climbing? Thats ...,NaN,Midwest
3,8,#WEATHER: 10:53 am : 63.0F. Feels 61F. 30.07% ...,tennessee,"Nashville, TN, USA"
4,12,We used to use umbrellas to face the bad weath...,NaN,Houston


In [31]:
### use twokenized data
twokenized_train = ps.read_fwf('train_tokenized.txt',header=None)
train['tweet'] = twokenized_train[0]
twokenized_test = []
twokenized_test = ps.read_fwf('ark-tweet-nlp-0.3.2/test_tokenized.txt',header=None)
test['tweet'] = twokenized_test[0]

,id,tweet,state,location
0,4,Edinburgh peeps is it sunny ?? #weather,NaN,birmingham
1,5,""" SEEVERE T’STORM WARNING FOR TROUSDALE , NORT...",NaN,Nashville
2,7,@Agilis1 sport or traditional climbing ? Thats...,NaN,Midwest
3,8,#WEATHER : 10:53 am : 63.0 F . Feels 61F . 30....,tennessee,"Nashville, TN, USA"
4,12,""" We used to use umbrellas to face the bad wea...",NaN,Houston


In [32]:
tfidf = TfidfVectorizer(max_features=10000, strip_accents='unicode', analyzer='word', ngram_range=(1, 3))
tfidf.fit(train['tweet'])

# tfidf.get_feature_names()

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [34]:
X = tfidf.transform(train['tweet'])
y = np.array(train.iloc[:,4:])
X_test = tfidf.transform(test['tweet'].values.astype('U')) 

X_train, X_val, y_train, y_val = train_test_split(X, y)

print(y_train)

[[0.    0.    0.801 ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.801 ... 0.    0.    0.   ]
 ...
 [0.195 0.21  0.    ... 0.506 0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]]


In [35]:
clf = Ridge(alpha=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

In [36]:
rmse = math.sqrt(mean_squared_error(y_val, y_pred))
rmse
1:0.16278244422812435

2:0.16274669578608342

0.16245072122626003

# Test Submission

In [16]:
test_prediction = clf.predict(X_test)

In [17]:
prediction = np.array(np.hstack([np.matrix(test['id']).T, test_prediction])) 
col = '%i,' + '%f,'*23 + '%f'
np.savetxt('data/output.txt', prediction,col, delimiter=',')

In [18]:
print('Train error: {0}'.format(np.sqrt(np.sum(np.array(np.array(clf.predict(X))-y)**2)/ (X.shape[0]*24.0))))



Train error: 0.1443787633511532
